In [1]:
import sys
sys.path.append('/home/potzschf/repos/')
from helperToolz.helpsters import *
import rasterio
from rasterio.merge import merge
import numpy as np
import glob


base_path = '/data/Aldhani/eoagritwin/et/Landsat/LST_44/'

# File paths
files = getFilelist(base_path + 'raw_scenes/', 'TIF', deep=True)
st_paths = [file for file in files if 'ST_B10' in file]
qa_paths = [file for file in files if 'QA_PIXEL' in file]



In [31]:
for idx in range(len(st_paths)):
    #if st_paths[idx].split('/')[-1].split('_B10')[0] == qa_paths[idx].split('/')[-1].split('_QA')[0]:
    if st_paths[idx].split('/')[-1].split('_ST_B10')[0] == qa_paths[idx].split('/')[-1].split('_QA')[0]:
        out_path = base_path + 'masked_scenes/' + st_paths[idx].split('/')[-1].split('_B10')[0] + 'masked.tif'

        # Open LST band
        with rasterio.open(st_paths[idx]) as src:
            st = src.read(1).astype("float32")
            profile = src.profile

        # Open QA band
        with rasterio.open(qa_paths[idx]) as src:
            qa = src.read(1)

        
        # Apply mask
        st_masked = np.where(qa == 21824, st, np.nan)
        st_masked_scaled = (st_masked * 0.00341802) + 149.0
        # Update profile for output
        profile.update(dtype=rasterio.float32, nodata=np.nan)

        # Write masked LST to new GeoTIFF
        with rasterio.open(out_path, "w", **profile) as dst:
            dst.write(st_masked_scaled.astype(np.float32), 1)

        print(f"Masked LST saved to {out_path}")

    else:
        print('sth is off')

Masked LST saved to /data/Aldhani/eoagritwin/et/Landsat/LST_44/masked_scenes/LC08_L2SP_191023_20190712_20200827_02_T1_STmasked.tif
Masked LST saved to /data/Aldhani/eoagritwin/et/Landsat/LST_44/masked_scenes/LC08_L2SP_191024_20190712_20200827_02_T2_STmasked.tif
Masked LST saved to /data/Aldhani/eoagritwin/et/Landsat/LST_44/masked_scenes/LC08_L2SP_193022_20190710_20200827_02_T1_STmasked.tif
Masked LST saved to /data/Aldhani/eoagritwin/et/Landsat/LST_44/masked_scenes/LC08_L2SP_193023_20190710_20200827_02_T1_STmasked.tif
Masked LST saved to /data/Aldhani/eoagritwin/et/Landsat/LST_44/masked_scenes/LC08_L2SP_193023_20190726_20200827_02_T1_STmasked.tif
Masked LST saved to /data/Aldhani/eoagritwin/et/Landsat/LST_44/masked_scenes/LC08_L2SP_193024_20190710_20200827_02_T1_STmasked.tif
Masked LST saved to /data/Aldhani/eoagritwin/et/Landsat/LST_44/masked_scenes/LC08_L2SP_195022_20190708_20200827_02_T1_STmasked.tif
Masked LST saved to /data/Aldhani/eoagritwin/et/Landsat/LST_44/masked_scenes/LC08_L

In [2]:
# examine image from July 26 2019
img = '/data/Aldhani/eoagritwin/et/Landsat/LST_44/masked_scenes/LC08_L2SP_193023_20190726_20200827_02_T1_STmasked.tif'
vrt_files = [file for file in getFilelist('/data/Aldhani/eoagritwin/et/Sentinel3/LST/LST_values/sharpened2/vrt/', '.vrt', deep=True) 
             if all(substr in file for substr in ['July_26', 'S2notMasked'])]

vrt_warps = []
for vrt_file in vrt_files:
    # warp_raster_to_reference(vrt_file,
    #                         img,
    #                         f'{base_path}warped_to_Landsat/{vrt_file.split('.vrt')[0].split('/')[-1]}_warped.tif',
    #                         resampling='average')
    vrt_warps.append(f'{base_path}warped_to_Landsat/{vrt_file.split('.vrt')[0].split('/')[-1]}_warped.tif')



# warp_raster_to_reference('/data/Aldhani/eoagritwin/et/Auxiliary/DEM/reprojected/THUENEN_GER_LST_WARP_BINARY.tif',
#                         img,
#                         f'{base_path}warped_to_Landsat/THUENEN_2021_bin_warped.tif',
#                         resampling='nearest')



In [3]:
# extract and export
landsat = gdal.Open(img)
landsat_arr = landsat.GetRasterBand(1).ReadAsArray()

# mask = gdal.Open(f'{base_path}/warped_to_Landsat/THUENEN_2021_bin_warped.tif')
# mask_arr = mask.GetRasterBand(1).ReadAsArray()

# landsat_arr_masked = np.where(np.logical_and(mask_arr == 1, landsat_arr != 149), landsat_arr, np.nan)
landsat_arr_masked = np.where(landsat_arr != 149, landsat_arr, np.nan)

for vrt_warp in vrt_warps:
    s2 = gdal.Open(vrt_warp)
    s2_arr = s2.GetRasterBand(1).ReadAsArray()
    s2_arr[s2_arr == 0] = np.nan
    if landsat_arr_masked.shape == s2_arr.shape:
        mask_extr = np.logical_and(~np.isnan(landsat_arr_masked),~np.isnan(s2_arr))
    else:
        print('sth messed up')
        continue
    landsat_values = landsat_arr_masked[mask_extr]
    s2_values = s2_arr[mask_extr]

    # Create a DataFrame
    df = pd.DataFrame({
        "Landsat": landsat_values,
        "Sentinel2": s2_values
    })

    # Export to CSV
    df.to_csv(f"{base_path}extracts/extract_{vrt_warp.split('.tif')[0].split('/')[-1]}_no_agromask.csv")

/data/Aldhani/users/potzschf/conda/envs/workhorse/lib/python3.12/site-packages/osgeo/gdal.py:311: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


In [ ]:
from rasterio.warp import calculate_default_transform, reproject, Resampling

files = [file for file in getFilelist(base_path, '.tif') if 'masked_strict' in file]
dst_crs = "EPSG:32633"  # choose your reference CRS

for f in files:
    with rasterio.open(f) as src:
        transform, width, height = calculate_default_transform(
            src.crs, dst_crs, src.width, src.height, *src.bounds
        )
        profile = src.profile.copy()
        profile.update(crs=dst_crs, transform=transform, width=width, height=height)

        out_file = f.replace(".tif", "_reproj.tif")
        with rasterio.open(out_file, "w", **profile) as dst:
            reproject(
                source=rasterio.band(src, 1),
                destination=rasterio.band(dst, 1),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=dst_crs,
                resampling=Resampling.nearest
            )

In [ ]:
src_files = [rasterio.open(file) for file in getFilelist(base_path, '.tif') if '_reproj' in file]


# Use rasterio.merge.merge to get mosaic and transform
mosaic, out_trans = merge(src_files, method='max')  # 'max' handles overlapping pixels

# mosaic is a 3D array with shape (1, height, width)
mosaic = mosaic[0]  # drop first dimension

# Use the profile from the first raster as a template
profile = src_files[0].profile
profile.update(
    height=mosaic.shape[0],
    width=mosaic.shape[1],
    transform=out_trans,
    dtype=rasterio.float32,
    nodata=np.nan
)

# Save the mosaic
out_file = f"{base_path}landsat_max_mosaic.tif"
with rasterio.open(out_file, "w", **profile) as dst:
    dst.write(mosaic.astype(np.float32), 1)

# Close all datasets
for src in src_files:
    src.close()

In [ ]:
warp_raster_to_reference(f'{base_path}landsat_max_mosaic.tif',
                         '/data/Aldhani/eoagritwin/et/Sentinel3/LST/LST_values/sharpened/vrt/July_09_allpred_short_minVZA_July_09_S2notMasked_withoutLSTmask.vrt',
                         f'{base_path}landsat_max_mosaic_warped.tif')

In [ ]:
warp_raster_to_reference('/data/Aldhani/eoagritwin/et/Sentdatedatedaddffdsfsdfsdfsdfsdsdfsdfaaahkhjkkjhkjhkjhkjhkjhkkjh/LST/LST_values/sharpened/vrt/July_09_allpred_short_minVZA_July_09_S2notMasked_withoutLSTmask.vrt',
                         f'{base_path}landsat_max_mosaic.tif',
                         f'{base_path}S2_vrt_warped.tif',
                         resampling='average')

Raster warped and saved to: /data/Aldhani/eoagritwin/et/Landsat/LST_44/S2_vrt_warped.tif


In [ ]:
landsat = gdal.Open(f'{base_path}landsat_max_mosaic_warped.tif')
landsat_arr = landsat.GetRasterBand(1).ReadAsArray()
s2 = gdal.Open('/data/Aldhani/eoagritwin/et/Sentinel3/LST/LST_values/sharpened/vrt/July_09_allpred_short_minVZA_July_09_S2notMasked_withoutLSTmask.vrt')
s2_arr = s2.GetRasterBand(1).ReadAsArray()

if landsat_arr.shape == s2_arr.shape:
    mask = np.logical_and(~np.isnan(landsat_arr),~np.isnan(s2_arr))

landsat_values = landsat_arr[mask]
s2_values = s2_arr[mask]

# Create a DataFrame
df = pd.DataFrame({
    "Landsat": landsat_values,
    "Sentinel2": s2_values
})

# Export to CSV
df.to_csv(f'{base_path}extract.csv')

In [5]:
landsat = gdal.Open(f'{base_path}landsat_max_mosaic.tif')
landsat_arr = landsat.GetRasterBand(1).ReadAsArray()
s2 = gdal.Open(f'{base_path}S2_vrt_warped.tif')
s2_arr = s2.GetRasterBand(1).ReadAsArray()

if landsat_arr.shape == s2_arr.shape:
    mask = np.logical_and(~np.isnan(landsat_arr),~np.isnan(s2_arr))

landsat_values = landsat_arr[mask]
s2_values = s2_arr[mask]

# Create a DataFrame
df = pd.DataFrame({
    "Landsat": landsat_values,
    "Sentinel2": s2_values
})

# Export to CSV
df.to_csv(f'{base_path}extract_S2_resampled.csv')